In [5]:
import facebook_business
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.targetingsearch import TargetingSearch
import pandas as pd
import csv
import matplotlib.pyplot as plt

In [ ]:
# API access token and secret app id
my_app_id = ''
my_app_secret = ''
my_access_token = ''
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)
my_account = AdAccount('')

In [ ]:
'''
The function get_delivery_estimate() only takes city ids instead of city names. 
TargetingSearchTypes.geolocation extracts the corresponding key ids.
'''

cities = pd.read_csv("C:/Data in the wild/EV_data/EV_Project-main/trends_data.csv")    
cities_list = list(pd.unique(cities['City']))

# get the parameters for each city. 
params = {}
for i in range(len(cities_list)):
    params[i] = {
        'q': cities_list[i],
        'location_types': ["city"],
        'type': TargetingSearch.TargetingSearchTypes.geolocation,
        'status': 'PAUSED'
    }

# change 'Copenhagen' to 'København'
params[2]['q'] = 'København'

In [ ]:
'''
TargetingSearch is computationally heavy so the id keys for each of the Danish cities are saved in a CSV.
'''
resp = []
for i in range(len(params)):
    if TargetingSearch.search(params[i]):
        for k in range(len(TargetingSearch.search(params[i]))):
            if TargetingSearch.search(params=params[i])[k]['country_code'] == 'DK':
                resp.append([TargetingSearch.search(params=params[i])[k]['key'],
                             TargetingSearch.search(params=params[i])[k]['name'],
                             TargetingSearch.search(params=params[i])[k]['region'],
                             TargetingSearch.search(params=params[i])[k]['region_id']])
    else:
        pass


# write the list of city ids and names to a csv file to avoid long computation
with open('City_id_and_name.txt', "w", newline="") as x:
    csv.writer(x, delimiter=",").writerows(resp)


In [ ]:
'''
The targeting dictionary contains all the specifications for the targeted audience.
The specification below targets Facebook users aged 18-65 living in a selection of cities whom Facebook
presume are interested in the topic 'Elbil'/'Electric vehicle'.
'''

#Code to get daily or monthly users in a specific targeting category
targeting = {}
for i in range(len(resp)):
    targeting[i] = {
            'geo_locations': {
                #'countries': ['DK'],
                'cities':[ 
                    {'key': resp[i][0],
                     #'radius': '10',
                     #'distance_unit': 'mile'
                     }]},
            'age_min': 18,
            'age_max': 65,
            'interests': [
                {
                    'id': 6003103779434,
                    'name': 'Elbil' ,
                },
            ],
        }


In [ ]:
'''
Param contains the targeting specifications in the fitting dictionary structure to be 
used in get_delivery_estimate()
'''

# specify parameters for get_delivery_estimate()
param = {}
for i in range(len(targeting)):
    param[i] = {
        'targeting_spec': targeting[i],
        'optimization_goal': 'REACH',
        #'daily_budget': '50',
        'status': 'PAUSED'
    }


# save the parameters as a dictionary to avoid exceeding call limit.
np.save('/paramters_for_deliveryestimate_v3.npy', param)

In [ ]:
import numpy as np
import time
# Load the parameters
param = np.load('paramters_for_deliveryestimate_v3.npy',allow_pickle='TRUE').item()

# Use get_delivery on the specified parameters. Sleep is used beause the  API 
# has a limit of 300 calls per hour. 
delivery_estimate = []
for i in range(len(param)):
    delivery_estimate.append(my_account.get_delivery_estimate(params = param[i]))
    time.sleep(15)

# make a list of only city ID + upper and lower bound of monthly active users 
estimates = []
for i in range(len(delivery_estimate)):
    estimates.append([(delivery_estimate[i].params['targeting_spec']['geo_locations']['cities'][0]['key']), 
                      int(delivery_estimate[i][0]['estimate_mau_lower_bound']),
                      int(delivery_estimate[i][0]['estimate_mau_upper_bound'])])

# write the estimates as a CSV
with open('FB_estimates_121221.txt', "w", newline="") as x:
    csv.writer(x, delimiter=",").writerows(estimates)


In [ ]:
# Match city id with city name, clean and name columns and write to csv. 
# get_delivery_estimate() only outputs FBs city Ids.

City_id_and_name = pd.read_csv("/City_id_and_name.txt", header = None, encoding='cp1252')
FB_estimates_121221 = pd.read_csv("/FB_estimates_121221.txt", header = None)

fb_estimates = pd.merge(City_id_and_name, FB_estimates_121221, on= 0)
fb_estimates = fb_estimates.drop_duplicates(subset = 0)

fb_estimates = fb_estimates.rename(columns={0: 'City id', '1_x': 'City', 
                                            '2_x': 'Region', 3: 'Region id',
                                            '1_y': 'Lower bound', '2_y': 'Upper bound'})

fb_estimates.to_csv("FB_estimates.csv")